# 1). Set Up the Environment

In [2]:
#import base libraries
import pandas as pd

In [3]:
# import train datasets
google_review_data = pd.read_json("review-Vermont_10.json", lines=True)
advertisements_data = pd.read_json("advertisements.json")
irrelevant_data = pd.read_json("irrelevant.json")
rant_data = pd.read_json("rant_without_visit.json")

# 2). Exploratory Data Analysis

2a). Visualise the Google Review dataset...

In [4]:
print("\n----------FIRST FIVE INSTANCES IN GOOGLE REVIEW----------\n")
display(google_review_data.head())
print("\n----------GOOGLE REVIEW DATA INFORMATION----------\n")
display(google_review_data.info())
print("\n----------GOOGLE REVIEW DATA SUMMARY STATS----------\n")
display(google_review_data.describe())


----------FIRST FIVE INSTANCES IN GOOGLE REVIEW----------



,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.180269e+20,rebecca kerns,1620085852324,5,Always done right from wood stove to screens!,None,"{'time': 1620087641504, 'text': 'Good Evening,...",0x89e02445cb9db457:0x37f42bff4edf7a43
1,1.015327e+20,Peter DeForge,1580309946474,5,A great company to work with. Their sales and...,None,"{'time': 1580320228721, 'text': 'Good Afternoo...",0x89e02445cb9db457:0x37f42bff4edf7a43
2,1.154041e+20,Chad Goulette,1605195974445,5,Great place to do business with staff was grea...,None,"{'time': 1605195166792, 'text': 'Hi Chad! Tha...",0x89e02445cb9db457:0x37f42bff4edf7a43
3,1.047893e+20,Mark LaFountain,1593005848256,5,"Awesome Customer service, quick response, and ...",None,"{'time': 1593376422014, 'text': 'Mark, thank y...",0x89e02445cb9db457:0x37f42bff4edf7a43
4,1.089807e+20,Jeff R,1582059996120,5,"If you need a top quality job, by a group of p...",None,"{'time': 1582063833737, 'text': 'Good Afternoo...",0x89e02445cb9db457:0x37f42bff4edf7a43



----------GOOGLE REVIEW DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324725 entries, 0 to 324724
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  324725 non-null  float64
 1   name     324725 non-null  object 
 2   time     324725 non-null  int64  
 3   rating   324725 non-null  int64  
 4   text     178168 non-null  object 
 5   pics     9651 non-null    object 
 6   resp     26838 non-null   object 
 7   gmap_id  324725 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 19.8+ MB


None


----------GOOGLE REVIEW DATA SUMMARY STATS----------



,user_id,time,rating
count,3.247250e+05,3.247250e+05,324725.000000
mean,1.091867e+20,1.548171e+12,4.314346
std,5.313166e+18,3.945905e+10,0.996339
min,1.000018e+20,1.182960e+12,1.000000
25%,1.045707e+20,1.523215e+12,4.000000
50%,1.091955e+20,1.550804e+12,5.000000
75%,1.137058e+20,1.573925e+12,5.000000
max,1.184442e+20,1.631106e+12,5.000000



Key Takeaways

- Columns with missing data in the train dataset: text, pics and resp
- Train dataset contains 324725 examples with 8 variables

2b). Visualise the advertisements, irrelevant, and rant_without_visit datasets...

In [5]:
print("\n----------FIRST FIVE INSTANCES IN ADVERTISEMENTS----------\n")
display(advertisements_data.head())
print("\n----------ADVERTISEMENTS DATA INFORMATION----------\n")
display(advertisements_data.info())
print("\n----------ADVERTISEMENTS DATA SUMMARY STATS----------\n")
display(advertisements_data.describe())

print("\n----------FIRST FIVE INSTANCES IN IRRELEVANT----------\n")
display(irrelevant_data.head())
print("\n----------IRRELEVANT DATA INFORMATION----------\n")
display(irrelevant_data.info())
print("\n----------IRRELEVANT DATA SUMMARY STATS----------\n")
display(irrelevant_data.describe())

print("\n----------FIRST FIVE INSTANCES IN RANT----------\n")
display(rant_data.head())
print("\n----------RANT DATA INFORMATION----------\n")
display(rant_data.info())
print("\n----------RANT DATA SUMMARY STATS----------\n")
display(rant_data.describe())


----------FIRST FIVE INSTANCES IN ADVERTISEMENTS----------



,reviews
0,{'text': 'The food at Luigi's Pizzeria was dec...
1,{'text': 'I was at Oak Creek Dental for a clea...
2,{'text': 'The handyman at City Hardware helped...
3,{'text': 'Stopped by The Daily Grind Cafe for ...
4,{'text': 'The new tires I got from Pete's Auto...



----------ADVERTISEMENTS DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  151 non-null    object
dtypes: object(1)
memory usage: 1.3+ KB


None


----------ADVERTISEMENTS DATA SUMMARY STATS----------



...

,reviews
0,{'text': 'Absolutely the worst place ever. I w...
1,{'text': 'I heard from a friend of a friend ab...
2,{'text': 'After reading some things online abo...
3,{'text': 'The service is notoriously awful. Ev...
4,{'text': 'I drove by this place and just from ...



----------RANT DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  150 non-null    object
dtypes: object(1)
memory usage: 1.3+ KB


None


----------RANT DATA SUMMARY STATS----------



,reviews
count,150
unique,150
top,{'text': 'Absolutely the worst place ever. I w...
freq,1



Key Takeaways

- Advertisements, Irrelevant and Rant datasets contain 151, 150, and 150 examples respectively.
- No duplicates in all datasets

# 3). Preprocessing and Modeling 

In [6]:
import ollama
import re
from sklearn.metrics import precision_score, recall_score, f1_score

3a). Preprocessing

In [7]:
genuine_texts = [item for item in google_review_data[:240]["text"] if item is not None]
# According to deepseek r1, these reviews are all genuine

ad_reviews_list = advertisements_data["reviews"]
ad_texts = [review["text"] for review in ad_reviews_list]

irre_reviews_list = irrelevant_data["reviews"]
irre_texts = [review["text"] for review in irre_reviews_list]

rant_reviews_list = rant_data["reviews"]
rant_texts = [review["text"] for review in rant_reviews_list]

# Construct our evaluation dataset
columns=['text', 'ad_score', 'irre_score', 'rant_score']
data = []
for text in genuine_texts[:120]:
    data.append([text, 0, 0, 0])
for text in ad_texts[:120]:
    data.append([text, 1, 0, 0])
for text in irre_texts[:120]:
    data.append([text, 0, 1, 0])
for text in rant_texts[:120]:
    data.append([text, 0, 0, 1])
evaluation_df = pd.DataFrame(data, columns=columns).sample(frac=1.0, random_state=42)
X, y = evaluation_df["text"], evaluation_df.drop(columns=["text"])

# Retrieve examples for few-shot learning
print("genuine review example:", genuine_texts[121])
print("advertisement review example:", ad_texts[121])
print("irrelevant review example:", irre_texts[121])
print("rant without visit review example:", rant_texts[121])

# Visualise Evaluation Set
print("\n----------FIRST FIVE INSTANCES IN EVALUATION SET----------\n")
display(evaluation_df.head())
print("\n----------EVALUATION SET DATA INFORMATION----------\n")
display(evaluation_df.info())
print("\n----------EVALUATION SET DATA SUMMARY STATS----------\n")
display(evaluation_df.describe())

genuine review example: Unique use of renovated nursing home. Artists in residence,  on site restaurant and small meeting room space.
advertisement review example: The bartender at The Rusty Nail noticed my new watch. It's a luxury replica from LuxuryReplicasDirect.com. It looks and feels 100% real but costs 90% less. For this week only, free shipping on all orders over $150. Visit their site and use code BAR10 for an extra 10% off.
irrelevant review example: I think everyone should learn to play a musical instrument. It's good for the soul. I'm trying to learn the ukulele.
rant without visit review example: The stories I've heard are enough to make me never want to go within a mile of this place. It sounds like a truly hellish experience.

----------FIRST FIVE INSTANCES IN EVALUATION SET----------



,text,ad_score,irre_score,rant_score
73,A nice easy walk in the woods.,0,0,0
414,Zero stars if I could. This business is a blig...,0,0,1
394,A complete and total dump. I can't believe the...,0,0,1
277,The new update for my phone's operating system...,0,1,0
399,The most incompetent staff I have ever had the...,0,0,1



----------EVALUATION SET DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 73 to 102
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        480 non-null    object
 1   ad_score    480 non-null    int64 
 2   irre_score  480 non-null    int64 
 3   rant_score  480 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 18.8+ KB


None


----------EVALUATION SET DATA SUMMARY STATS----------



,ad_score,irre_score,rant_score
count,480.000000,480.000000,480.000000
mean,0.250000,0.250000,0.250000
std,0.433464,0.433464,0.433464
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.250000,0.250000,0.250000
max,1.000000,1.000000,1.000000


3b). Evaluating Model: `gemma3:12b`

In [33]:
# Prompting and Prediction

batch_size = 1 # Let the model classify multiple reviews at a time for increased speed
model = 'gemma3:12b'

i = 0
pred_data = []
columns = ['ad_score', 'irre_score', 'rant_score']
y_pred = pd.DataFrame(pred_data, columns=columns)

# Start with the initial prompt with few-shot learning
content = """
You are an expert review classifier. Your task is to analyze user reviews and assign a confidence level for three specific categories: "advertisement", "irrelevant", and "rant without visit".

**Categories & Examples:**
-   **advertisement**: "Best pizza! Visit www.pizzapromo.com for discounts!"
-   **irrelevant**: "I love my new phone, but this place is too noisy."
-   **rant without visit**: "Never been here, but i heard it's terrible."

**Input:**
A list of reviews.

**Task:**
For the provided reviews, analyze the text to determine a confidence level (Low, or High) for each category.

**Output Instructions:**
- **DO NOT** include any other text, explanations, or commentary before or after the JSON.
- **DO NOT** include a `reviews` or any other key for the main JSON array.
- The output must be a single JSON array `[]` containing one JSON object `{}` for each review. All reviews must be analysed.
- Each object must have exactly three keys: `"advertisement"`, `"irrelevant"`, and `"rant without visit"`.
- Each value must be one of the strings: `"Low"` or `"High"`.
- The format must be identical to the example output.

**Example Input:**
"Unique use of renovated nursing home. Artists in residence,  on site restaurant and small meeting room space.", "The bartender at The Rusty Nail noticed my new watch. It's a luxury replica from LuxuryReplicasDirect.com. It looks and feels 100% real but costs 90% less. For this week only, free shipping on all orders over $150. Visit their site and use code BAR10 for an extra 10% off.", "I think everyone should learn to play a musical instrument. It's good for the soul. I'm trying to learn the ukulele.", "The stories I've heard are enough to make me never want to go within a mile of this place. It sounds like a truly hellish experience."

**Example Output:**
```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "High",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "High",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "High"
  }
]
"""
messages=[
    {'role': 'user', 'content': content}
]

# 2. Make the first call
response1 = ollama.chat(model=model, messages=messages)
print(response1['message']['content'])

# 3. Append the model's response to the message history
messages.append({'role': 'assistant', 'content': response1['message']['content']})

while i < len(evaluation_df): #len(evaluation_df)
  print(f"Processing reviews {str(i+1)} to {str(i+batch_size)} out of {len(evaluation_df)}")
  # 4. Add the next user prompt to test reviews
  content2 = ", ".join([f'"{item}"' for item in X[i:i+batch_size]])
  messages.append({'role': 'user', 'content': content2})

  # 5. Make the second call, passing the entire updated message list
  response2 = ollama.chat(model=model, messages=messages)
  messages.pop() # remove the previous message

  actual_output = response2['message']['content']
  print(actual_output)

  raw_pred = []
  words_to_find = ["advertisement", "irrelevant", "rant without visit"]
  word_count_ineq = False
  for word in words_to_find:
      word_count = 0
      pattern = r'"' + word + r'":'
      for match in re.finditer(pattern, actual_output, re.IGNORECASE):
          print(f"Found '{match.group()}' at index {match.start()}")
          print(actual_output[match.end():match.end()+10].lower())
          if "low" in actual_output[match.end():match.end()+10].lower():
              raw_pred.append(0)
          if "high" in actual_output[match.end():match.end()+10].lower():
              raw_pred.append(1)
          word_count += 1
      if word_count != batch_size:
          print(f"word_count expected {batch_size} but got {word_count}")
          break
      
  if word_count_ineq:
      continue

  if len(raw_pred) != 3*batch_size:
      print(f"length incorrect, expected {3*batch_size} but got {len(raw_pred)}")
      continue

  columns = ['ad_score', 'irre_score', 'rant_score']
  for j in range(batch_size):
      y_pred.loc[len(y_pred)] = [raw_pred[j], raw_pred[j+batch_size], raw_pred[j+2*batch_size]]

  i += batch_size

```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "High",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "High",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "High"
  }
]
```
Processing reviews 1 to 1 out of 480
```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  }
]
```
Found '"advertisement":' at index 18
 "low",
  
Found '"irrelevant":' at index 46
 "low",
  
Found '"rant without visit":' at index 71
 "low"
  }
Processing reviews 2 to 2 out of 480
```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "High"
  }
]
```
Found '"advertisement":' at index 18
 "low",
  
Found '"irrelevant":' at index 46
 "low",
  
Found '"rant without visit":' at index 71
 "high"
  
Processing reviews 3 

In [ ]:
# Results

y_truth = y

for category in y_truth.columns:
    print(f"--- Metrics for {category} ---")

    true_labels = y_truth[category]
    pred_labels = y_pred[category]

    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1:.3f}\n")

"""
batch size: 4

--- Metrics for ad_score ---
Precision: 0.992
Recall: 0.992
F1-Score: 0.992

--- Metrics for irre_score ---
Precision: 0.889
Recall: 0.867
F1-Score: 0.878

--- Metrics for rant_score ---
Precision: 0.931
Recall: 0.792
F1-Score: 0.856

Execution Time: 15m 57.2s
"""

"""
batch size: 1

--- Metrics for ad_score ---
Precision: 0.976
Recall: 1.000
F1-Score: 0.988

--- Metrics for irre_score ---
Precision: 0.922
Recall: 0.883
F1-Score: 0.902

--- Metrics for rant_score ---
Precision: 0.925
Recall: 0.925
F1-Score: 0.925

Execution Time 19m 49.1s
"""

--- Metrics for ad_score ---
Precision: 0.976
Recall: 1.000
F1-Score: 0.988

--- Metrics for irre_score ---
Precision: 0.922
Recall: 0.883
F1-Score: 0.902

--- Metrics for rant_score ---
Precision: 0.925
Recall: 0.925
F1-Score: 0.925



'\nbatch size: 4\n\n--- Metrics for ad_score ---\nPrecision: 0.992\nRecall: 0.992\nF1-Score: 0.992\n\n--- Metrics for irre_score ---\nPrecision: 0.889\nRecall: 0.867\nF1-Score: 0.878\n\n--- Metrics for rant_score ---\nPrecision: 0.931\nRecall: 0.792\nF1-Score: 0.856\n\nExecution Time: 15m 57.2s\n'

3c). Evaluating Model: `qwen3:8b`

In [ ]:
# Prompting and Prediction

batch_size = 8 # Let the model classify multiple reviews at a time for increased speed
model = 'qwen3:8b'

i = 0
pred_data = []
columns = ['ad_score', 'irre_score', 'rant_score']
y_pred = pd.DataFrame(pred_data, columns=columns)

# Start with the initial prompt with few-shot learning
content = """
You are an expert review classifier. Your task is to analyze user reviews and assign a confidence level for three specific categories: "advertisement", "irrelevant", and "rant without visit".

**Categories & Examples:**
-   **advertisement**: "Best pizza! Visit www.pizzapromo.com for discounts!"
-   **irrelevant**: "I love my new phone, but this place is too noisy."
-   **rant without visit**: "Never been here, but i heard it's terrible."

**Input:**
A list of reviews.

**Task:**
For the provided reviews, analyze the text to determine a confidence level (Low, or High) for each category.

**Output Instructions:**
- **DO NOT** include any other text, explanations, or commentary before or after the JSON.
- **DO NOT** include a `reviews` or any other key for the main JSON array.
- The output must be a single JSON array `[]` containing one JSON object `{}` for each review. All reviews must be analysed.
- Each object must have exactly three keys: `"advertisement"`, `"irrelevant"`, and `"rant without visit"`.
- Each value must be one of the strings: `"Low"` or `"High"`.
- The format must be identical to the example output.

**Example Input:**
Great place to do business with staff was great,thank u!", "The new burger joint is a hit. I could only enjoy it because my heartburn is gone thanks to AcidFix tablets. Get a free trial bottle at GutHealthNow.net (just pay s&h). You can eat anything without worry!", "I think reality TV has peaked and is now in a steady decline. The golden age is over.", "I don't know how they sleep at night knowing how badly they operate. The lack of care is astounding. Everything is a mess."

**Example Output:**
```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "High",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "High",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "High"
  }
]
"""
messages=[
    {'role': 'user', 'content': content}
]

# 2. Make the first call
response1 = ollama.chat(model=model, messages=messages)
print(response1['message']['content'])

# 3. Append the model's response to the message history
messages.append({'role': 'assistant', 'content': response1['message']['content']})

while i < len(evaluation_df): #len(evaluation_df)
  print(f"Processing reviews {str(i+1)} to {str(i+batch_size)} out of {len(evaluation_df)}")
  # 4. Add the next user prompt to test reviews
  content2 = ", ".join([f'"{item}"' for item in X[i:i+batch_size]])
  messages.append({'role': 'user', 'content': content2})

  # 5. Make the second call, passing the entire updated message list
  response2 = ollama.chat(model=model, messages=messages)
  messages.pop() # remove the previous message

  index = response2['message']['content'].find("</think>") + len("</think>")
  actual_output = response2['message']['content'][index:]
  print(actual_output)

  raw_pred = []
  words_to_find = ["advertisement", "irrelevant", "rant without visit"]
  word_count_ineq = False
  for word in words_to_find:
      word_count = 0
      pattern = r'"' + word + r'":'
      for match in re.finditer(pattern, actual_output, re.IGNORECASE):
          print(f"Found '{match.group()}' at index {match.start()}")
          print(actual_output[match.end():match.end()+10].lower())
          if "low" in actual_output[match.end():match.end()+10].lower():
              raw_pred.append(0)
          if "high" in actual_output[match.end():match.end()+10].lower():
              raw_pred.append(1)
          word_count += 1
      if word_count != batch_size:
          print(f"word_count expected {batch_size} but got {word_count}")
          word_count_ineq = True
          break
      
  if word_count_ineq == True:
      continue

  if len(raw_pred) != 3*batch_size:
      print(f"length incorrect, expected {3*batch_size} but got {len(raw_pred)}")
      continue

  columns = ['ad_score', 'irre_score', 'rant_score']
  for j in range(batch_size):
      y_pred.loc[len(y_pred)] = [raw_pred[j], raw_pred[j+batch_size], raw_pred[j+2*batch_size]]

  i += batch_size

<think>
Okay, let's tackle this review classification task. The user wants me to assign confidence levels for three categories: advertisement, irrelevant, and rant without visit. Each review needs to be checked against these categories with either Low or High confidence. 

First, I need to understand the definitions and examples given. 

For "advertisement", the example is a review that includes a promotional link or a direct offer. So any text that has a clear promotional intent, like mentioning a website for discounts, would be advertisement. 

"Irrelevant" refers to reviews that mention something unrelated to the main topic. The example is a user talking about their phone while reviewing a place, which is off-topic. So if a review shifts to a different subject, that's irrelevant. 

"Rant without visit" is when someone hasn't visited the place but is expressing strong negative opinions. The example is someone saying they never been there but heard it's terrible. 

Now, looking at the

In [ ]:
# Results

y_truth = y

for category in y_truth.columns:
    print(f"--- Metrics for {category} ---")

    true_labels = y_truth[category]
    pred_labels = y_pred[category]

    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1:.3f}\n")

"""
--- Metrics for ad_score ---
Precision: 0.976
Recall: 1.000
F1-Score: 0.988

--- Metrics for irre_score ---
Precision: 0.833
Recall: 0.750
F1-Score: 0.789

--- Metrics for rant_score ---
Precision: 0.911
Recall: 0.600
F1-Score: 0.724


Execution Time 44m 28.5s
"""

--- Metrics for ad_score ---
Precision: 0.976
Recall: 1.000
F1-Score: 0.988

--- Metrics for irre_score ---
Precision: 0.833
Recall: 0.750
F1-Score: 0.789

--- Metrics for rant_score ---
Precision: 0.911
Recall: 0.600
F1-Score: 0.724



'\n\n'

# 4). Discussion

## 4a). Summarize findings

Gemma 3 demonstrated superior overall performance, achieving higher F1-scores across all three violation categories while being significantly faster (over 2.8x) than Qwen 3. Both models excelled at detecting advertisements but showed a noticeable performance gap on the more nuanced tasks of identifying irrelevant content and rants without visit, with Gemma 3 maintaining a stronger balance between precision and recall.

For Gemma 3, we also compared batch processing performance and single processing performance. 
- Single processing yields significantly higher accuracy for the most challenging categories:

    - Rant Detection (F1-Score): Increased from 0.856 to 0.925 (+8.1%). This is the most substantial gain, highlighting that identifying unsubstantiated rants requires the model's full, undivided attention on a single text.

    - Irrelevant Content Detection: Also improved from 0.878 to 0.902.

- Batch processing reviews with a batch size of 4 is approximately 24% faster (15m 57s vs. 19m 49s). 

- The performance for detecting straightforward Advertisement reviews remained near-perfect (F1-Score ~0.99) regardless of batch size

## 4b). Limitations

Limitation: Absence of Ground Truth and Synthetic Label Generation

The Issue: The provided dataset contained only genuine reviews. Labels and examples for the violation categories (Advertisement, Irrelevant Content, Rant Without Visit) had to be synthetically generated using DeepSeek R1, a more advanced LLM.

Why it's a Limitation:

- Introduction of Bias: The synthetic data will inherently reflect the biases, styles, and knowledge cut-off of DeepSeek R1. The model may generate stereotypical, "textbook" examples of violations that lack the nuance and creativity of real-world human-written violations.

- Lack of Real-World Noise: Real data is messy. Synthetic data often lacks the spelling errors, grammatical mistakes, cultural references, and subtle contextual cues that characterize genuine spam and rants. A model trained on "clean" synthetic data may struggle with the "noise" of real data.

- Circular Validation: There is a risk of a "closed loop." Using an LLM (DeepSeek R1) to generate data to test other LLMs (Gemma 3, Qwen 3) may artificially inflate performance metrics if the models share similar architectural biases or training data. The evaluation does not test performance on real human violations, but on a different AI's simulation of them.

## 4c). Suggested Improvements

Improvement: Leverage Geographical and Business Context

Incorporate metadata such as the user's GPS coordinates (from mobile app usage), the business's physical address (from its gmap_id), and the business name and category into the given dataset.

How it improves model detection rate:

- "Rant Without Visit" Detection: A review from a user whose GPS history shows they have never been within 50 miles of the business is an enormous red flag. This single feature would drastically improve recall for this category.

- "Irrelevant Content" Detection: Knowing the business category (e.g., "auto repair") allows the model to more accurately judge if a review about "smart home devices" is off-topic.

Implementation Challenge: Converting gmap_id requires the Google Places API, which is a paid service. The cost must be weighed against the value of improved moderation. For a large-scale application, this would be a necessary and justified operational cost.

## 4d). Future Work

Multi-Stage Moderation Pipeline

For Example:

- Stage 1: Rule-Based Filtering. Instantly catch obvious violations using cheap regex patterns (e.g., URLs, phone numbers, profanity word lists).

- Stage 2: Lightweight ML Model. Use a fast, classical model (e.g., CatBoost, XGBoost) trained on TF-IDF (Term Frequency-Inverse Document Frequency), count, linguistic and metadata features.

- Stage 3: Heavyweight LLM. Only the most ambiguous reviews that pass the first two stages are sent to the expensive LLM (Gemma 3) for a final decision.

This Pipeline can reduce latency and cut costs, making the solution viable for real-world deployment.

# 5. Demo

5a). Initialize the model 

In [60]:
import ollama
import re
model = 'gemma3:12b'

# Start with the initial prompt with few-shot learning
content = """
You are an expert review classifier. Your task is to analyze user reviews and assign a confidence level for three specific categories: "advertisement", "irrelevant", and "rant without visit".

**Categories & Examples:**
-   **advertisement**: "Best pizza! Visit www.pizzapromo.com for discounts!"
-   **irrelevant**: "I love my new phone, but this place is too noisy."
-   **rant without visit**: "Never been here, but i heard it's terrible."

**Input:**
A list of reviews.

**Task:**
For the provided reviews, analyze the text to determine a confidence level (Low, or High) for each category.

**Output Instructions:**
- **DO NOT** include any other text, explanations, or commentary before or after the JSON.
- **DO NOT** include a `reviews` or any other key for the main JSON array.
- The output must be a single JSON array `[]` containing one JSON object `{}` for each review. All reviews must be analysed.
- Each object must have exactly three keys: `"advertisement"`, `"irrelevant"`, and `"rant without visit"`.
- Each value must be one of the strings: `"Low"` or `"High"`.
- The format must be identical to the example output.

**Example Input:**
"Unique use of renovated nursing home. Artists in residence,  on site restaurant and small meeting room space.", "The bartender at The Rusty Nail noticed my new watch. It's a luxury replica from LuxuryReplicasDirect.com. It looks and feels 100% real but costs 90% less. For this week only, free shipping on all orders over $150. Visit their site and use code BAR10 for an extra 10% off.", "I think everyone should learn to play a musical instrument. It's good for the soul. I'm trying to learn the ukulele.", "The stories I've heard are enough to make me never want to go within a mile of this place. It sounds like a truly hellish experience."

**Example Output:**
```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "High",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "High",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "High"
  }
]
"""
messages=[
    {'role': 'user', 'content': content}
]

# 2. Make the first call
response1 = ollama.chat(model=model, messages=messages)

# 3. Append the model's response to the message history
messages.append({'role': 'assistant', 'content': response1['message']['content']})

5b). Classify Your Review (Single)

In [61]:
# Insert your review to classify here
# e.g. review = "I heard from my friends that everything is bad!"
review = "I heard from my friends that everything is bad!"

In [62]:
# Let the model classify your review
content2 = review
messages.append({'role': 'user', 'content': content2})

response2 = ollama.chat(model=model, messages=messages)
messages.pop()

llm_output = response2['message']['content']

words_to_find = ["advertisement", "irrelevant", "rant without visit"]
outputs = []
for word in words_to_find:
    pattern = r'"' + word + r'":'
    for match in re.finditer(pattern, llm_output, re.IGNORECASE):
        if "low" in llm_output[match.end():match.end()+10].lower():
            outputs.append(0)
        if "high" in llm_output[match.end():match.end()+10].lower():
            outputs.append(1)

if outputs[0] == 0 and outputs[1] == 0 and outputs[2] == 0:
    print("This review is: Genuine")
elif outputs[0] == 1:
    print("This review is: Advertisement")
elif outputs[1] == 1:
    print("This review is: Irrelevant")
elif outputs[2] == 1:
    print("This review is: Rant Without Visit")

This review is: Rant Without Visit


5c). Classify Your Reviews (Batch)

In [65]:
# Insert your file containing reviews to classify here
reviews = "test_reviews.txt"

In [66]:
# Let the model classify all the reviews in the file
batch_size = 4

review_list = []

with open(reviews, 'r') as file:
    for line in file:
        review_list.append(line.strip())

i = 0
while i < len(review_list):
  print(f"Processing reviews {str(i+1)} to {str(min(i+batch_size, len(review_list)))} out of {len(review_list)}")
  content2 = ", ".join([f'"{item}"' for item in review_list[i:min(i+batch_size, len(review_list))]])
  messages.append({'role': 'user', 'content': content2})

  response2 = ollama.chat(model=model, messages=messages)
  messages.pop()

  llm_output = response2['message']['content']

  raw_pred = []
  words_to_find = ["advertisement", "irrelevant", "rant without visit"]
  word_count_ineq = False
  for word in words_to_find:
      word_count = 0
      pattern = r'"' + word + r'":'
      for match in re.finditer(pattern, llm_output, re.IGNORECASE):
          if "low" in llm_output[match.end():match.end()+10].lower():
              raw_pred.append(0)
          if "high" in llm_output[match.end():match.end()+10].lower():
              raw_pred.append(1)
          word_count += 1
      if word_count != min(batch_size, len(review_list)-i):
          print(f"word_count expected {min(batch_size, len(review_list)-i)} but got {word_count}")
          break
      
  if word_count_ineq:
      continue

  if len(raw_pred) != 3*min(batch_size, len(review_list)-i):
      print(f"length incorrect, expected {3*min(batch_size, len(review_list)-i)} but got {len(raw_pred)}")
      continue

  columns = ['ad_score', 'irre_score', 'rant_score']
  for j in range(min(batch_size, len(review_list)-i)):
        print("Review:", review_list[i+j])
        if raw_pred[j]:
            print("This review is: Advertisement")
        elif raw_pred[j + min(batch_size, len(review_list)-i)]:
            print("This review is: Irrelevant")
        elif raw_pred[j + 2*min(batch_size, len(review_list)-i)]:
            print("This review is: Rant without visit")
        else:
            print("This review is: Genuine")
        print('\n')

  i += batch_size

Processing reviews 1 to 4 out of 4
Review: Really nice people, nice selection of Lyndon Furniture, exactly the table I wanted at a reasonable price!
This review is: Genuine


Review: I was at the gym and my personal trainer asked me about my pre-workout. I told him about NitroFuel XT. It gives you insane energy with no crash. They're running a promo: buy two tubs, get a third one free. Go to nitrofuelxt.com and use code GYMFLOW. You'll break all your personal records.
This review is: Advertisement


Review: The squirrels in my backyard are getting too bold. One almost took my entire bird feeder the other day. Need a better strategy.
This review is: Irrelevant


Review: The entitlement and rudeness are off the charts. I can't believe they treat people the way they do. Absolutely appalling behavior.
This review is: Rant without visit


